In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

In [2]:
cam = 2
fps = 240

model_version = "my_model_13.0.keras"
Im_Width = 128
Im_Height = 128

In [3]:
def crop_image(frame, x, y, w, h):
    return frame[y:y+h, x:x+w]

In [4]:
def measure_brightness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    brightness = np.mean(gray)
    return brightness 

In [5]:
def measure_edge_richness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    edges = cv2.Canny(gray, 200, 400)  # Apply Canny edge detection
    edge_richness = np.sum(edges)  # Sum of edge pixel intensities
    return edge_richness

In [6]:
def select_best_frame(frame_buffer):

    best_frame = None
    highest_brightness = 50
    highest_edge_richness = 150000

    for frame in frame_buffer:

        #Crop
        cropped_frame = crop_image(frame, 220, 0, 200, 200)

        # Measure brightness
        brightness = measure_brightness(cropped_frame)

        if brightness > 200:
            # Update the best frame if this one is better
            if brightness > highest_brightness:
                highest_brightness = brightness
                best_frame = frame  # Save the frame as the best frame
    
        elif brightness > 50 and brightness < 100:
            # Apply edge detection
            edge_richness = measure_edge_richness(cropped_frame)
            # Update the best frame if this one is better
            if edge_richness > highest_edge_richness:
                highest_edge_richness = edge_richness
                best_frame = frame  # Save the cropped frame as the best frame

    return best_frame  # Returns None if no frames meet the threshold


In [7]:
def process_frame(frame, target_size):

    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

In [8]:
def predict_frame(frame, model, target_size):

    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

    # Make a prediction
    predictions = model.predict(frame_array)  # Outputs probabilities for each class
    predicted_class = np.argmax(predictions[0])  # Get the index of the class with the highest probability
    confidence = predictions[0][predicted_class]  # Confidence score for the predicted class

    # Map class indices to labels
    class_labels = {0: "Defective", 2: "Non-Defective", 1: "No Bottle"}
    predicted_label = class_labels[predicted_class]
    
    return predicted_label, confidence

In [9]:
def capture(cam, fps, model_path, target_size=(Im_Width, Im_Height)):

    cap = cv2.VideoCapture(cam)  # Open webcam
    # Set desired frame rate
    desired_fps = fps
    #cap.set(cv2.CAP_PROP_FPS, desired_fps)

    frame_buffer = []
    max_frames = 15  # Number of frames to evaluate

    # Load the model
    print("Loading model...")
    model = load_model(model_path)
    print("Model loaded successfully.")

    while True:

        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Exiting...")
            break

        frame_buffer.append(frame)

        # Process the buffer when it's full
        if len(frame_buffer) == max_frames:
            best_frame = select_best_frame(frame_buffer)
            frame_buffer = []  # Clear the buffer

            # Process and predict
            if best_frame is not None:
                feedback, confidence = predict_frame(best_frame, model, target_size)
                print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

                cv2.putText(best_frame, f"Prediction: {feedback}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(best_frame, f"Confidence: {confidence:.2f}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
                # Display the best frame
                cv2.imshow("Best Frame", best_frame)
            '''
            else:
                image_path = "no-bottle.bmp"
                image = cv2.imread(image_path)
                # Display the image in a window
                cv2.putText(best_frame, f"No Bottle!", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow("Pre-Saved Image", image)'''

            # Exit on 'ESC' key
            if cv2.waitKey(1) & 0xFF == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

In [10]:
if __name__ == "__main__":
    model_path = model_version  # Path to your saved model
    capture(cam, fps, model_path, target_size=(Im_Width, Im_Height))

Loading model...
Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Prediction: No Bottle (Confidence: 0.45)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Prediction: No Bottle (Confidence: 0.47)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Prediction: Defective (Confidence: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Prediction: Defective (Confidence: 0.99)
